## Configuration of cairio.org

Right now the central cairio server is hosted on https://pairio.org:20443

This is a linode server

Inside a tmux session called cairio:

```
cd ~/src/spikeforest2/mountaintools/cairioserver
```

See .env file containing:

```
PORT=20443
CAIRIO_ADMIN_TOKEN=******
```

To run the server:

```
npm install .
node cairioserver.js
```

There are instructions in the home directory for setting up firewall and renewing the certs

In [1]:
%load_ext autoreload
%autoreload 2

# import cairio client and other packages
from mountaintools import client as mt
from cairio import CairioClient
from getpass import getpass
import os
from getpass import getpass
import json
from simplecrypt import encrypt, decrypt
import base64
from dotenv import load_dotenv

Loading environment from: /home/magland/.mountaintools/.env


In [3]:
# Load secret stuff from the cairio.env file (only on the admin's computer)

load_dotenv(dotenv_path='cairio.env',verbose=True)
CAIRIO_ADMIN_TOKEN=os.environ['CAIRIO_ADMIN_TOKEN']

In [5]:
# Let's set up the admin collection where we will store the user login information
# It will have the same token as the admin token

mt.addRemoteCollection(collection='admin', token=CAIRIO_ADMIN_TOKEN, admin_token=CAIRIO_ADMIN_TOKEN)

True

In [6]:
# Now let's add some users.

def set_user_config(user,config):
    key=dict(
        name='user_config',
        user=user
    )
    val=base64.b64encode(encrypt(getpass('Enter an encryption password for {}: '.format(user)),json.dumps(config))).decode()
    cc=CairioClient()
    cc.setRemoteConfig(collection='admin',token=CAIRIO_ADMIN_TOKEN)
    cc.setValue(
        key=key,
        value=val
    )

# user=magland
config=dict(
    kbucket_shares=[
        dict(hub='kbucket.flatironinstitute.org',name='sf_raw',node_id='15734439d8cf'),
        dict(hub='kbucket.flatironinstitute.org',name='spikeforest1',node_id='7317cea8265b',upload_token=os.environ['SPIKEFOREST1_UPLOAD_TOKEN']),
        dict(hub='kbucket.flatironinstitute.org',name='spikeforest2',node_id='69432e9201d0',upload_token=os.environ['SPIKEFOREST2_UPLOAD_TOKEN'])
    ],
    cairio_collections=[
        dict(database='default',name='spikeforest',token=os.environ['SPIKEFOREST_TOKEN'])
    ]
)
set_user_config('magland',config)
set_user_config('jjun',config)

Enter an encryption password for magland:  ·····


CAIRIO CONFIG: remote database admin (readwrite); local sha-1 cache


Enter an encryption password for jjun:  ·····


CAIRIO CONFIG: remote database admin (readwrite); local sha-1 cache


In [8]:
# Now this is how we would log in as magland and configure the cairio client:

mt.login(user='magland', ask_password=True)
mt.configRemoteReadWrite(collection='spikeforest',share_id='spikeforest.spikeforest2')

Mountain password for magland:  ·····


Logging in as magland...
Logged in as magland
CAIRIO CONFIG: remote database spikeforest (readwrite); remote kb-share 69432e9201d0 (readwrite)


In [10]:
# Or we can configure for readonly

mt.configRemoteReadonly(collection='spikeforest',share_id='spikeforest.spikeforest2')

CAIRIO CONFIG: remote database spikeforest (readonly); remote kb-share 69432e9201d0 (readonly)


In [11]:
# Let's set the token for the remote spikeforest collection

collection='spikeforest'
mt.addRemoteCollection(collection=collection, token=os.environ['SPIKEFOREST_TOKEN'], admin_token=CAIRIO_ADMIN_TOKEN)

True

In [2]:
###### Let's do a test
mt.login(user='magland',ask_password=True)
mt.configRemoteReadWrite(collection='spikeforest',share_id='spikeforest.spikeforest2')

mt.saveObject(key='test1',object=dict(test=1))
print(mt.loadObject(key='test1'))
mt.saveObject(key='test1',object=dict(test=2))
print(mt.loadObject(key='test1'))

Mountain password for magland:  ·····


Logging in as magland...
Logged in as magland
CAIRIO CONFIG: remote database spikeforest (readwrite); remote kb-share 69432e9201d0 (readwrite)
{'test': 1}
{'test': 2}


In [3]:
# Let's make an alias so we don't need to remember the share_id
mt.setValue(key='spikeforest2',value='69432e9201d0')

True

In [5]:
# Now we can do this!
mt.configRemoteReadWrite(collection='spikeforest',share_id='spikeforest.spikeforest2')

Resolved share_id 69432e9201d0 from alias spikeforest.spikeforest2
CAIRIO CONFIG: remote database spikeforest (readwrite); remote kb-share 69432e9201d0 (readwrite)
